In [48]:
import pandas as pd
import nltk

df = pd.read_csv('review-text-equal-count.csv')

In [66]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [28]:
alphabet = list('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ 1234567890')

def clean_text(txt:str):
    # People who write with ellipses?
    txt=str(txt)
    txt.replace('(Translated by Google)', '')

    # Replace other characters
    for char in txt:
        if char not in alphabet:
            txt = txt.replace(char, '')
            
    return txt.lower()

In [68]:
df['cleaned'] = df['text'].apply(clean_text)

In [69]:
def tokenize(txt):
    return nltk.word_tokenize(str(txt))

df['tokenized'] = df['cleaned'].apply(tokenize)

In [74]:
df.sample(12)

,rating,text,cleaned,tokenized
59687,2,They never get a order right,they never get a order right,"[they, never, get, a, order, right]"
76671,2,"So they get 2 stars for the nice, clean, fresh...",so they get 2 stars for the nice clean freshly...,"[so, they, get, 2, stars, for, the, nice, clea..."
149344,3,Service and food are ok.,service and food are ok,"[service, and, food, are, ok]"
212602,5,one of the great gems of Pawtucket,one of the great gems of pawtucket,"[one, of, the, great, gems, of, pawtucket]"
223218,5,LOVE THIS PLACE,love this place,"[love, this, place]"
149787,3,Ok place but dirty,ok place but dirty,"[ok, place, but, dirty]"
84061,2,Service was efficient. Food was not even warm....,service was efficient food was not even warm o...,"[service, was, efficient, food, was, not, even..."
106944,3,"Good service and dish variety, but not very no...",good service and dish variety but not very not...,"[good, service, and, dish, variety, but, not, ..."
179291,4,Excellent help from the counter staff.,excellent help from the counter staff,"[excellent, help, from, the, counter, staff]"
241305,5,Love it. My dog has a great time,love it my dog has a great time,"[love, it, my, dog, has, a, great, time]"


In [124]:
from nltk.corpus import stopwords

stop_words = list(stopwords.words('english')) 

In [141]:
word_dict = {}
i = 0

for tokens in df['tokenized']:
    for word in tokens:
        if (word in word_dict) or (word in stop_words):
            continue
            
        else: 
            word_dict[word] = i
            i += 1

word_dict_inv = {v:k for k, v in word_dict.items()}

In [153]:
def vectorize(tokens):
    vec = []
    for word in tokens:
        if word in stop_words:
            continue
            
        val = word_dict[word]
        if val in vec:
            continue
            
        else: 
            vec.append(word_dict[word])
            
    return vec
    
df['vectorized'] = df['tokenized'].apply(vectorize)      


In [177]:
df_ct = df[['rating', 'text', 'tokenized']].copy()

df_orig = df[['rating', 'text']]

df_vec = df[['rating', 'text', 'vectorized']]

In [137]:
import pickle

filename = "word_dict.pkl"

# Save the dictionary
with open(filename, 'wb') as file:
    pickle.dump(word_dict, file)

In [157]:
df.to_csv('review-text-equal-count.csv', index=False)